<a href="https://colab.research.google.com/github/JuanDiaz77/Proyecto-colab/blob/main/Interpretaci%C3%B3n_de_modelos_con_LIME_en_%C3%A1rboles_de_decisi%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===========================================
# 1️ Instalación de librerías necesarias
# ===========================================

!pip install lime -q

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from lime.lime_tabular import LimeTabularExplainer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ===========================================
# 2️ Cargar dataset (ejemplo: breast cancer)
# ===========================================

data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

print(" Dataset cargado correctamente")
print(f"Número de muestras: {X.shape[0]}")
print(f"Número de variables: {X.shape[1]}")
print("\nDistribución de clases (0=Maligno, 1=Benigno):")
print(y.value_counts())

# ===========================================
# 3️ Preprocesamiento
# ===========================================

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División de los datos
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# ===========================================
# 4️ Entrenar modelo (RandomForest)
# ===========================================

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("\n Reporte de clasificación:")
print(classification_report(y_test, y_pred))
print(f"Exactitud del modelo: {accuracy_score(y_test, y_pred):.3f}")

# ===========================================
# 5️ Configurar LIME
# ===========================================

explainer = LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=data.feature_names,
    class_names=data.target_names,
    mode='classification'
)

# Elegimos una instancia para explicar (por ejemplo, la primera del test)
i = 0
exp = explainer.explain_instance(
    X_test[i],
    model.predict_proba,
    num_features=10
)

# ===========================================
# 6️ Mostrar explicación visual
# ===========================================

exp.show_in_notebook(show_table=True, show_all=False)

# Si no estás en notebook, también puedes usar:
# exp.as_pyplot_figure()
# plt.show()

# ===========================================
# 7️ Interpretación textual
# ===========================================

print("\n Explicación de LIME para la muestra", i)
for feature, weight in exp.as_list():
    print(f"{feature}: {weight:.4f}")

